### Library and model imports 

In [1]:
# general module imports
from optlang.symbolics import Zero, add
import json
import os
import re

# import the thermodynamic API
import thermodynamic_API_KBase_01

# KBase model import
os.environ["HOME"] = 'C:\\Users\\Andrew Freiburger\\Dropbox\\My PC (DESKTOP-M302P50)\\Documents\\UVic Civil Engineering\\Internships\\Agronne\\cobrakbase'
import cobrakbase
token = 'SF3CW6PH4CKWSRMOA3JQPSGC6YERTRSF'
kbase = cobrakbase.KBaseAPI(token)

# remove duplicate compounds from the KBase model 
object_json = kbase.get_object('E_iAH991V2', 93832)
unique_mcs = dict((x['id'], x) for x in object_json['modelcompounds']) 
object_json['modelcompounds'] = list(unique_mcs.values()) # update data without the duplicate id
kbase.save_object('E_iAH991V2', 'freiburgermsu:narrative_1624557251879', 'KBaseFBA.FBAModel', object_json) # saving the object back to KBase with id=E_iAH991V2 to the workspace freiburgermsu:narrative_1624557251879 assigning this type KBaseFBA.FBAModel and with data=object_json

# import the refined KBase individual model
model = kbase.get_from_ws('E_iAH991V2', 93832)

# locally import modelseed
import modelseedpy
from modelseedpy.biochem import from_local
modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
modelseed = modelseedpy.biochem.from_local(modelseed_path)

'''# import the E. coli BiGG COBRA model
import cobra
bigg_model_path = '..\COBRA function scripts\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)'''

cobrakbase 0.2.8


"# import the E. coli BiGG COBRA model\nimport cobra\nbigg_model_path = '..\\COBRA function scripts\\e_coli_core metabolism from BiGG.json'\nmodel = cobra.io.load_json_model(bigg_model_path)"

## Creating a sample thermodynamic dataset  

A sample dataset is created that will be passed through the following example

In [2]:
from scipy.constants import calorie
from pandas import DataFrame

reactions = {}
undescribed_compounds = set()
for reaction in object_json['modelreactions']:
    reagents = {}
    reaction_id = reaction['id']
    reaction_name = reaction['name']
    stoich = reaction['modelReactionReagents']
    for reagent in stoich:
        # parse the ModelSEED compound id
        modelseed_id = re.search('(?<=\/)([^\/]+$)', reagent['modelcompound_ref']).group(1)
        
        # parse the ModelSEED compound compartment and refine the ModelSEED ID
        compartment = re.search('(_[a-z][0-9])', modelseed_id).group(1)
        compartment = re.sub('(_)', '', compartment)
        modelseed_id = re.sub('(_[a-z][0-9])', '', modelseed_id)
        
        #print(modelseed_id)
        if re.search('cpd', modelseed_id):
            for key, value in modelseed.get_seed_compound(modelseed_id).data.items():
                if key == 'charge':
                    compound_charge = value
                elif key == 'deltag':
                    compound_gibbs = value / calorie
        
        else:
            print('ERROR: The {} compound is undescribed by the ModelSEED database'.format(modelseed_id))
            undescribed_compounds.add(modelseed_id)
            compound_charge = ''
            compound_gibbs = ''
        
        # create the reagent dictionary
        if modelseed_id not in reagents:
            reagents[modelseed_id] = {'coefficient': reagent['coefficient'], 
                                      'compartment': compartment,
                                      'charge': compound_charge,
                                      'gibbs (KJ/mol)': compound_gibbs}

        elif modelseed_id in reagents:
            reagents[modelseed_id].update({'coefficient_2': reagent['coefficient'], 
                                           'compartment_2': compartment,
                                           'charge_2': compound_charge,
                                           'gibbs_2 (KJ/mol)': compound_gibbs})
            
    # create the reaction dictionary
    reactions[reaction_id] = {'name': reaction_name,
                              'reagents': reagents}
    
'''with open('2021-06-30_undescribed compounds from the E_iAH991V2 individual model.csv', 'w') as output:
    DataFrame(undescribed_compounds, columns = ['undescribed']).to_csv(output)'''

# review the contents of the data parsing  
'''print(list(reactions.values())[-1])
for reagent, information in reagents.items():
    print(reagent, ': ', information)'''

ERROR: The 12ethd compound is undescribed by the ModelSEED database
ERROR: The 12ethd compound is undescribed by the ModelSEED database
ERROR: The 2ameph compound is undescribed by the ModelSEED database
ERROR: The pald compound is undescribed by the ModelSEED database
ERROR: The 2fuclac compound is undescribed by the ModelSEED database
ERROR: The 2mbutACP compound is undescribed by the ModelSEED database
ERROR: The 2mmal compound is undescribed by the ModelSEED database
ERROR: The 2mmale compound is undescribed by the ModelSEED database
ERROR: The 2mmale compound is undescribed by the ModelSEED database
ERROR: The e3mmal compound is undescribed by the ModelSEED database
ERROR: The 3fuclac compound is undescribed by the ModelSEED database
ERROR: The 3hgACPm compound is undescribed by the ModelSEED database
ERROR: The egACPm compound is undescribed by the ModelSEED database
ERROR: The epACPm compound is undescribed by the ModelSEED database
ERROR: The 3hpACPm compound is undescribed by 

"print(list(reactions.values())[-1])\nfor reagent, information in reagents.items():\n    print(reagent, ': ', information)"

# Base FBA functionality

The Base FBA class introduces the essential functions that are inherited into the following functions.  

In [3]:
announcement = 'BaseFBApkg'
print(announcement, '\n', '='*len(announcement))


#--------------------------- __init__ ---------------------------------
print('\n+ __init__')

base = thermodynamic_API_KBase_01.BaseFBAPkg(model = model, name = 'test_model')    
    
print('model used: ', base.model)
print('instance name: ', base.name)
print('variable types: ', base.variable_types)
print('constraint types: ', base.constraint_types)


#-------------------------- validate_parameters -----------------------
print('\n+ validate_parameters')

params = {'a':2, 'b':4, 'c':3}
required = ['a', 'b', 'c']
defaults = {'a':1, 'b':1, 'c':1, 'd':1}

print('parameters before: ', base.parameters)
base.validate_parameters(params, required, defaults)
print('parameters after: ', base.parameters)


#-------------------------- build_variable -----------------------------
print('\n+ build_variable')
print('variables before: ', base.variables)

base.build_variable(kind = 'concentration', lower_bound = 10, upper_bound = 133, vartype = 'continuous', obj = model.reactions.get_by_id('12ETHDt_c0'))

print('variables after: ', base.variables)


#--------------------------- build_constraint --------------------------
print('\n+ build_constraint')
print('constraints before: ', base.constraints)

base.build_constraint(constraint_expression = 0, kind = 'concentration', lower_bound = 10, upper_bound = 133, obj = model.reactions.get_by_id('12ETHDt_c0'), coef = {})

print('constraints after: ', base.constraints)


#---------------------------- all_variables -----------------------------
print('\n+ all_variables')
print('model variables: ', base.all_variables())


#--------------------------- all_constraints ----------------------------
print('\n+ all_constraints')
print('model constraints: ', base.all_constraints())


#--------------------------- revert_to_original --------------------------
print('\n+ revert_to_original')

model2 = kbase.get_from_ws('E_iAH991V2', 93832)

variables = []
constraints = []
for var in model.variables:
    variables.append(str(var))
for constraint in model.constraints:
    constraints.append(str(constraint))
    
variables2 = []
constraints2 = []
for var in model2.variables:
    variables2.append(str(var))
for constraint in model2.constraints:
    constraints2.append(str(constraint))
    
difference_variables = set(variables) - set(variables2)
print('added variables before: ', difference_variables)
difference_constraints = set(constraints) - set(constraints2)
print('added constraints before: ', difference_constraints)

model = kbase.get_from_ws('E_iAH991V2', 93832)

variables = []
constraints = []
for var in model.variables:
    variables.append(str(var))
for constraint in model.constraints:
    constraints.append(str(constraint))
    
variables2 = []
constraints2 = []
for var in model2.variables:
    variables2.append(str(var))
for constraint in model2.constraints:
    constraints2.append(str(constraint))
    
difference_variables = set(variables) - set(variables2)
print('added variables after: ', difference_variables)
difference_constraints = set(constraints) - set(constraints2)
print('added constraints after: ', difference_constraints)

BaseFBApkg 

+ __init__
model used:  Bacteroides_thetaiotaomicron_VPI-5482.fbamdl.23
instance name:  test_model
variable types:  {'concentration': 'float', 'lnconc': 'float'}
constraint types:  {'concentration': 'float'}

+ validate_parameters
parameters before:  {}
parameters after:  {'a': 2, 'b': 4, 'c': 3, 'd': 1}

+ build_variable
variables before:  {'concentration': {}, 'lnconc': {}}
variables after:  {'concentration': {'12ETHDt_c0': 10 <= 12ETHDt_c0_concentration <= 133}, 'lnconc': {}}

+ build_constraint
constraints before:  {'concentration': {}}
{}
constraints after:  {'concentration': {'12ETHDt_c0': <optlang.cplex_interface.Constraint object at 0x000002CE5230D608>}}

+ all_variables
model variables:  {'concentration': {'12ETHDt_c0': 10 <= 12ETHDt_c0_concentration <= 133}, 'lnconc': {}}

+ all_constraints
model constraints:  {'concentration': {'12ETHDt_c0': <optlang.cplex_interface.Constraint object at 0x000002CE5230D608>}}

+ revert_to_original


added variables before:  {'10 <= 12ETHDt_c0_concentration <= 133'}
added constraints before:  {'12ETHDt_c0_concentration: 10 <= 0 <= 133'}


added variables after:  set()
added constraints after:  set()


# RevBin functionality

A reaction is evaluated as a variable for the forward and backward fluxes. 

In [4]:
announcement = 'Revbin'
print('\n', announcement, '\n', '='*len(announcement))


#--------------------------- __init__ ---------------------------------
print('\n+ __init__')

revbin = thermodynamic_API_KBase_01.RevBinPkg(model, obj = model.reactions.get_by_id('12ETHDt_c0'))

print('model used: ', revbin.model)
print('instance name: ', revbin.name)
print('variable types: ', revbin.variable_types)
print('constraint types: ', revbin.constraint_types)


#------------------------- build_constraint ---------------------------
print('\n+ build_constraint')
print('constraints before: ', revbin.constraints)

revbin.build_constraint(obj = model.reactions.get_by_id('12ETHDt_c0'))

print('constraints after: ', revbin.constraints)


#------------------------- build_package ------------------------------
print('\n+ build_package')
constraints = []
for constraint in revbin.constraints:
    for datam in revbin.constraints[constraint]:
        constraints.append(str(datam))
print('constraints before: ', len(constraints))

revbin.build_package(filter = constraints)

constraints = []
for constraint in revbin.constraints:
    for datam in revbin.constraints[constraint]:
        constraints.append(str(datam))
print('constraints after: ', len(constraints))


#------------------------- revert_to_original --------------------------
model = revbin.revert_to_original(kbase = kbase, kbase_model_id = 'E_iAH991V2', workspace_id = 93832)


 Revbin 

+ __init__
model used:  Bacteroides_thetaiotaomicron_VPI-5482.fbamdl.23
instance name:  reversible binary
variable types:  {'revbin': 'reaction', 'forv': 'reaction', 'revv': 'reaction'}
constraint types:  {'revbinF': 'reaction', 'revbinR': 'reaction'}

+ build_constraint
constraints before:  {'revbinF': {}, 'revbinR': {}}
{0 <= 12ETHDt_c0_revbin <= 1: -1000, 0 <= 12ETHDt_c0 <= 1000: 1}
{0 <= 12ETHDt_c0_revbin <= 1: 1000, 0 <= 12ETHDt_c0_reverse_af890 <= 1000: 1}
constraints after:  {'revbinF': {'12ETHDt_c0': <optlang.cplex_interface.Constraint object at 0x000002CE67E8A848>}, 'revbinR': {'12ETHDt_c0': <optlang.cplex_interface.Constraint object at 0x000002CE67E8A988>}}

+ build_package
constraints before:  2
{0 <= 12PPDRt_c0_revbin <= 1: -1000, 0 <= 12PPDRt_c0 <= 1000: 1}
{0 <= 12PPDRt_c0_revbin <= 1: 1000, 0 <= 12PPDRt_c0_reverse_86883 <= 1000: 1}
{0 <= 12PPDt_c0_revbin <= 1: -1000, 0 <= 12PPDt_c0 <= 1000: 1}
{0 <= 12PPDt_c0_revbin <= 1: 1000, 0 <= 12PPDt_c0_reverse_bf31c <= 

{0 <= CBLAT_c0_revbin <= 1: -1000, 0 <= CBLAT_c0 <= 1000: 1}
{0 <= CBLAT_c0_revbin <= 1: 1000, 0 <= CBLAT_c0_reverse_99847 <= 1000: 1}
{0 <= CBPS_c0_revbin <= 1: -1000, 0 <= CBPS_c0 <= 1000: 1}
{0 <= CBPS_c0_revbin <= 1: 1000, 0 <= CBPS_c0_reverse_56d80 <= 0: 1}
{0 <= CCP_c0_revbin <= 1: -1000, 0 <= CCP_c0 <= 1000: 1}
{0 <= CCP_c0_revbin <= 1: 1000, 0 <= CCP_c0_reverse_c7f2c <= 0: 1}
{0 <= CD2abc1_c0_revbin <= 1: -1000, 0 <= CD2abc1_c0 <= 1000: 1}
{0 <= CD2abc1_c0_revbin <= 1: 1000, 0 <= CD2abc1_c0_reverse_209e3 <= 0: 1}
{0 <= CDPGHL_c0_revbin <= 1: -1000, 0 <= CDPGHL_c0 <= 1000: 1}
{0 <= CDPGHL_c0_revbin <= 1: 1000, 0 <= CDPGHL_c0_reverse_658ce <= 0: 1}
{0 <= CDPMEK_c0_revbin <= 1: -1000, 0 <= CDPMEK_c0 <= 1000: 1}
{0 <= CDPMEK_c0_revbin <= 1: 1000, 0 <= CDPMEK_c0_reverse_75fd9 <= 0: 1}
{0 <= CEPTC__DASH__BT_c0_revbin <= 1: -1000, 0 <= CEPTC__DASH__BT_c0 <= 1000: 1}
{0 <= CEPTC__DASH__BT_c0_revbin <= 1: 1000, 0 <= CEPTC__DASH__BT_c0_reverse_d013e <= 0: 1}
{0 <= CEPTE__DASH__BT_c0_revb

{0 <= Clt_c0_revbin <= 1: -1000, 0 <= Clt_c0 <= 1000: 1}
{0 <= Clt_c0_revbin <= 1: 1000, 0 <= Clt_c0_reverse_99d45 <= 1000: 1}
{0 <= Cuabc_c0_revbin <= 1: -1000, 0 <= Cuabc_c0 <= 1000: 1}
{0 <= Cuabc_c0_revbin <= 1: 1000, 0 <= Cuabc_c0_reverse_280b3 <= 0: 1}
{0 <= Cut1_c0_revbin <= 1: -1000, 0 <= Cut1_c0 <= 1000: 1}
{0 <= Cut1_c0_revbin <= 1: 1000, 0 <= Cut1_c0_reverse_0111e <= 0: 1}
{0 <= DADK_c0_revbin <= 1: -1000, 0 <= DADK_c0 <= 1000: 1}
{0 <= DADK_c0_revbin <= 1: 1000, 0 <= DADK_c0_reverse_43f3c <= 1000: 1}
{0 <= DADNt2r_c0_revbin <= 1: -1000, 0 <= DADNt2r_c0 <= 1000: 1}
{0 <= DADNt2r_c0_revbin <= 1: 1000, 0 <= DADNt2r_c0_reverse_0e802 <= 1000: 1}
{0 <= DAGK160_c0_revbin <= 1: -1000, 0 <= DAGK160_c0 <= 1000: 1}
{0 <= DAGK160_c0_revbin <= 1: 1000, 0 <= DAGK160_c0_reverse_425cb <= 0: 1}
{0 <= DALAt2r_c0_revbin <= 1: -1000, 0 <= DALAt2r_c0 <= 1000: 1}
{0 <= DALAt2r_c0_revbin <= 1: 1000, 0 <= DALAt2r_c0_reverse_291ad <= 1000: 1}
{0 <= DAPDC_c0_revbin <= 1: -1000, 0 <= DAPDC_c0 <= 1000

{0 <= GLYLEUPEPT1t_c0_revbin <= 1: -1000, 0 <= GLYLEUPEPT1t_c0 <= 1000: 1}
{0 <= GLYLEUPEPT1t_c0_revbin <= 1: 1000, 0 <= GLYLEUPEPT1t_c0_reverse_2c105 <= 0: 1}
{0 <= GLYMET1c_c0_revbin <= 1: -1000, 0 <= GLYMET1c_c0 <= 1000: 1}
{0 <= GLYMET1c_c0_revbin <= 1: 1000, 0 <= GLYMET1c_c0_reverse_99dd6 <= 1000: 1}
{0 <= GLYMETt_c0_revbin <= 1: -1000, 0 <= GLYMETt_c0 <= 1000: 1}
{0 <= GLYMETt_c0_revbin <= 1: 1000, 0 <= GLYMETt_c0_reverse_1bb57 <= 0: 1}
{0 <= GLYPHEHYi_c0_revbin <= 1: -1000, 0 <= GLYPHEHYi_c0 <= 1000: 1}
{0 <= GLYPHEHYi_c0_revbin <= 1: 1000, 0 <= GLYPHEHYi_c0_reverse_79513 <= 0: 1}
{0 <= GLYPHEt_c0_revbin <= 1: -1000, 0 <= GLYPHEt_c0 <= 1000: 1}
{0 <= GLYPHEt_c0_revbin <= 1: 1000, 0 <= GLYPHEt_c0_reverse_77348 <= 1000: 1}
{0 <= GLYPROPRO1c_c0_revbin <= 1: -1000, 0 <= GLYPROPRO1c_c0 <= 1000: 1}
{0 <= GLYPROPRO1c_c0_revbin <= 1: 1000, 0 <= GLYPROPRO1c_c0_reverse_dcf56 <= 1000: 1}
{0 <= GLYPROPRO1e_c0_revbin <= 1: -1000, 0 <= GLYPROPRO1e_c0 <= 1000: 1}
{0 <= GLYPROPRO1e_c0_revbin <=

{0 <= PGPP140_c0_revbin <= 1: -1000, 0 <= PGPP140_c0 <= 1000: 1}
{0 <= PGPP140_c0_revbin <= 1: 1000, 0 <= PGPP140_c0_reverse_49f8e <= 0: 1}
{0 <= PGPP160_c0_revbin <= 1: -1000, 0 <= PGPP160_c0 <= 1000: 1}
{0 <= PGPP160_c0_revbin <= 1: 1000, 0 <= PGPP160_c0_reverse_c8a17 <= 0: 1}
{0 <= PGPP180_c0_revbin <= 1: -1000, 0 <= PGPP180_c0 <= 1000: 1}
{0 <= PGPP180_c0_revbin <= 1: 1000, 0 <= PGPP180_c0_reverse_f63c8 <= 0: 1}
{0 <= PGPP181_c0_revbin <= 1: -1000, 0 <= PGPP181_c0 <= 1000: 1}
{0 <= PGPP181_c0_revbin <= 1: 1000, 0 <= PGPP181_c0_reverse_a521a <= 0: 1}
{0 <= PGPPAI15_c0_revbin <= 1: -1000, 0 <= PGPPAI15_c0 <= 1000: 1}
{0 <= PGPPAI15_c0_revbin <= 1: 1000, 0 <= PGPPAI15_c0_reverse_63382 <= 0: 1}
{0 <= PGPPAI17_c0_revbin <= 1: -1000, 0 <= PGPPAI17_c0 <= 1000: 1}
{0 <= PGPPAI17_c0_revbin <= 1: 1000, 0 <= PGPPAI17_c0_reverse_b4d9f <= 0: 1}
{0 <= PGPPI14_c0_revbin <= 1: -1000, 0 <= PGPPI14_c0 <= 1000: 1}
{0 <= PGPPI14_c0_revbin <= 1: 1000, 0 <= PGPPI14_c0_reverse_574b8 <= 0: 1}
{0 <= PGPPI1

{0 <= EX_cpd00232_e0_revbin <= 1: -1000, 0 <= EX_cpd00232_e0 <= 1000: 1}
{0 <= EX_cpd00232_e0_revbin <= 1: 1000, 0 <= EX_cpd00232_e0_reverse_3983d <= 1000: 1}
{0 <= EX_cpd00058_e0_revbin <= 1: -1000, 0 <= EX_cpd00058_e0 <= 1000: 1}
{0 <= EX_cpd00058_e0_revbin <= 1: 1000, 0 <= EX_cpd00058_e0_reverse_34982 <= 1000: 1}
{0 <= EX_hspg_rest_e0_revbin <= 1: -1000, 0 <= EX_hspg_rest_e0 <= 1000: 1}
{0 <= EX_hspg_rest_e0_revbin <= 1: 1000, 0 <= EX_hspg_rest_e0_reverse_362be <= 1000: 1}
{0 <= EX_cpd00161_e0_revbin <= 1: -1000, 0 <= EX_cpd00161_e0 <= 1000: 1}
{0 <= EX_cpd00161_e0_revbin <= 1: 1000, 0 <= EX_cpd00161_e0_reverse_73b98 <= 1000: 1}
{0 <= EX_levan1000_e0_revbin <= 1: -1000, 0 <= EX_levan1000_e0 <= 1000: 1}
{0 <= EX_levan1000_e0_revbin <= 1: 1000, 0 <= EX_levan1000_e0_reverse_53f2e <= 1000: 1}
{0 <= EX_cpd00159_e0_revbin <= 1: -1000, 0 <= EX_cpd00159_e0 <= 1000: 1}
{0 <= EX_cpd00159_e0_revbin <= 1: 1000, 0 <= EX_cpd00159_e0_reverse_e1225 <= 1000: 1}
{0 <= EX_cpd00138_e0_revbin <= 1: -100

{0 <= EX_cpd15494_e0_revbin <= 1: -1000, 0 <= EX_cpd15494_e0 <= 1000: 1}
{0 <= EX_cpd15494_e0_revbin <= 1: 1000, 0 <= EX_cpd15494_e0_reverse_19140 <= 1000: 1}
{0 <= EX_hspg_degr_3_e0_revbin <= 1: -1000, 0 <= EX_hspg_degr_3_e0 <= 1000: 1}
{0 <= EX_hspg_degr_3_e0_revbin <= 1: 1000, 0 <= EX_hspg_degr_3_e0_reverse_e63ff <= 1000: 1}
{0 <= EX_glypro_e0_revbin <= 1: -1000, 0 <= EX_glypro_e0 <= 1000: 1}
{0 <= EX_glypro_e0_revbin <= 1: 1000, 0 <= EX_glypro_e0_reverse_1a294 <= 1000: 1}
{0 <= EX_rhamnogalurII_e0_revbin <= 1: -1000, 0 <= EX_rhamnogalurII_e0 <= 1000: 1}
{0 <= EX_rhamnogalurII_e0_revbin <= 1: 1000, 0 <= EX_rhamnogalurII_e0_reverse_488aa <= 1000: 1}
{0 <= EX_cpd00137_e0_revbin <= 1: -1000, 0 <= EX_cpd00137_e0 <= 1000: 1}
{0 <= EX_cpd00137_e0_revbin <= 1: 1000, 0 <= EX_cpd00137_e0_reverse_42e89 <= 1000: 1}
{0 <= EX_argarg_e0_revbin <= 1: -1000, 0 <= EX_argarg_e0 <= 1000: 1}
{0 <= EX_argarg_e0_revbin <= 1: 1000, 0 <= EX_argarg_e0_reverse_15c03 <= 1000: 1}
{0 <= EX_oligofru4_e0_revbin <

# Simple Thermo functionality

The thermodynamic perspectives of each object metabolite are determined. The class parameterizes each metabolite as a variable, and thereby presumes nothing of the metabolite thermodynamics.  

In [5]:
import thermodynamic_API_KBase_01
import re

model = kbase.get_from_ws('E_iAH991V2', 93832)
announcement = 'Simple Thermo'
print('\n', announcement, '\n', '='*len(announcement))


#--------------------------- __init__ ---------------------------------
print('\n+ __init__')
simple_thermo = thermodynamic_API_KBase_01.SimpleThermoPkg(model)
print('model used: ', simple_thermo.model)
print('instance name: ', simple_thermo.name)
print('variable types: ', simple_thermo.variable_types)
print('constraint types: ', simple_thermo.constraint_types)


#-------------------------- build_constraint ---------------------------
print('\n+ build_constraint')
print('constraints before: ', simple_thermo.constraints)
simple_thermo.build_constraint(obj = model.reactions.get_by_id('12ETHDt_c0'))
print('constraints after: ', simple_thermo.constraints)


#--------------------------- build_package -----------------------------
print('\n+ build_package')
before_constraints = 0
for kind, value in simple_thermo.constraints.items():
    for constraint in value:
        print(value[constraint])
        before_constraints += 1
print('constraints before: ', before_constraints)
    
simple_thermo.build_package(view_errors = False)

after_constraints = 0
for kind, value in simple_thermo.constraints.items():
    for constraint in value:
        print(value[constraint])
        after_constraints += 1
print('constraints after: ', after_constraints)    

#-------------------------- revert_to_original -------------------------
model = simple_thermo.revert_to_original(kbase = kbase, kbase_model_id = 'E_iAH991V2', workspace_id = 93832)


 Simple Thermo 

+ __init__
model used:  Bacteroides_thetaiotaomicron_VPI-5482.fbamdl.23
instance name:  simple thermo
variable types:  {'potential': 'metabolite', 'revbin': 'reaction', 'forv': {}, 'revv': {}}
constraint types:  {'simple_thermo': 'reaction', 'revbinF': {}, 'revbinR': {}}

+ build_constraint
constraints before:  {'simple_thermo': {}, 'revbinF': {}, 'revbinR': {}}
{0 <= 12ETHDt_c0_revbin <= 1: -1000, 0 <= 12ETHDt_c0 <= 1000: 1}
{0 <= 12ETHDt_c0_revbin <= 1: 1000, 0 <= 12ETHDt_c0_reverse_af890 <= 1000: 1}
{0 <= 12ETHDt_c0_revbin <= 1: 1000, 0 <= 12ethd_c0_potential <= 1000: 1, 0 <= 12ethd_e0_potential <= 1000: -1}
constraints after:  {'simple_thermo': {'12ETHDt_c0': <optlang.cplex_interface.Constraint object at 0x000002CE893E9A08>}, 'revbinF': {'12ETHDt_c0': <optlang.cplex_interface.Constraint object at 0x000002CE893E9808>}, 'revbinR': {'12ETHDt_c0': <optlang.cplex_interface.Constraint object at 0x000002CE893E9C08>}}

+ build_package
12ETHDt_c0_simple_thermo: 0 <= 1000.0

{0 <= AICART_c0_revbin <= 1: -1000, 0 <= AICART_c0 <= 1000: 1}
{0 <= AICART_c0_revbin <= 1: 1000, 0 <= AICART_c0_reverse_e2c5e <= 1000: 1}
{0 <= AICART_c0_revbin <= 1: 1000, 0 <= cpd02851_c0_potential <= 1000: -1, 0 <= cpd00201_c0_potential <= 1000: -1, 0 <= cpd00087_c0_potential <= 1000: 1, 0 <= cpd02884_c0_potential <= 1000: 1}
{0 <= AIHPDCOATA_c0_revbin <= 1: -1000, 0 <= AIHPDCOATA_c0 <= 1000: 1}
{0 <= AIHPDCOATA_c0_revbin <= 1: 1000, 0 <= AIHPDCOATA_c0_reverse_34876 <= 1000: 1}
{0 <= AIHPDCOATA_c0_revbin <= 1: 1000, 0 <= cpd11493_c0_potential <= 1000: 1, 0 <= aihpdcoa_c0_potential <= 1000: 1, 0 <= 14mpalmACP_c0_potential <= 1000: -1, 0 <= cpd00010_c0_potential <= 1000: -1}
{0 <= AIPDCOATA_c0_revbin <= 1: -1000, 0 <= AIPDCOATA_c0 <= 1000: 1}
{0 <= AIPDCOATA_c0_revbin <= 1: 1000, 0 <= AIPDCOATA_c0_reverse_6b989 <= 1000: 1}
{0 <= AIPDCOATA_c0_revbin <= 1: 1000, 0 <= cpd11493_c0_potential <= 1000: 1, 0 <= aipdcoa_c0_potential <= 1000: 1, 0 <= 12mmyrsACP_c0_potential <= 1000: -1, 0 <= c

{0 <= DASYNI17_c0_revbin <= 1: 1000, 0 <= DASYNI17_c0_reverse_8958a <= 0: 1}
{0 <= DASYNI17_c0_revbin <= 1: 1000, 0 <= cdpdihpdecg_c0_potential <= 1000: 1, 0 <= pai17_c0_potential <= 1000: -1, 0 <= cpd00012_c0_potential <= 1000: 1, 0 <= cpd00052_c0_potential <= 1000: -1, 0 <= cpd00067_c0_potential <= 1000: -1}
{0 <= DB4PS_c0_revbin <= 1: -1000, 0 <= DB4PS_c0 <= 1000: 1}
{0 <= DB4PS_c0_revbin <= 1: 1000, 0 <= DB4PS_c0_reverse_9efc3 <= 0: 1}
{0 <= DB4PS_c0_revbin <= 1: 1000, 0 <= cpd00047_c0_potential <= 1000: 1, 0 <= cpd11225_c0_potential <= 1000: 1, 0 <= cpd00067_c0_potential <= 1000: 1, 0 <= cpd00171_c0_potential <= 1000: -1}
{0 <= DBTS_c0_revbin <= 1: -1000, 0 <= DBTS_c0 <= 1000: 1}
{0 <= DBTS_c0_revbin <= 1: 1000, 0 <= DBTS_c0_reverse_691ab <= 0: 1}
{0 <= DBTS_c0_revbin <= 1: 1000, 0 <= cpd00008_c0_potential <= 1000: 1, 0 <= cpd00002_c0_potential <= 1000: -1, 0 <= cpd00067_c0_potential <= 1000: 3, 0 <= cpd00764_c0_potential <= 1000: -1, 0 <= cpd00011_c0_potential <= 1000: -1, 0 <= c

{0 <= G6PI3_c0_revbin <= 1: 1000, 0 <= cpd00072_c0_potential <= 1000: 1, 0 <= cpd00863_c0_potential <= 1000: -1}
{0 <= GACPCD_c0_revbin <= 1: -1000, 0 <= GACPCD_c0 <= 1000: 1}
{0 <= GACPCD_c0_revbin <= 1: 1000, 0 <= GACPCD_c0_reverse_f8b96 <= 0: 1}
{0 <= GACPCD_c0_revbin <= 1: 1000, 0 <= cpd11493_c0_potential <= 1000: 1, 0 <= cpd11492_c0_potential <= 1000: -1, 0 <= gACPm_c0_potential <= 1000: -1, 0 <= 3opACPm_c0_potential <= 1000: 1, 0 <= cpd00067_c0_potential <= 1000: -1, 0 <= cpd00011_c0_potential <= 1000: 1}
{0 <= GALAM6PDA_c0_revbin <= 1: -1000, 0 <= GALAM6PDA_c0 <= 1000: 1}
{0 <= GALAM6PDA_c0_revbin <= 1: 1000, 0 <= GALAM6PDA_c0_reverse_cfac8 <= 0: 1}
{0 <= GALAM6PDA_c0_revbin <= 1: 1000, 0 <= cpd00013_c0_potential <= 1000: 1, 0 <= cpd00001_c0_potential <= 1000: -1, 0 <= galam6p_c0_potential <= 1000: -1, 0 <= cpd00805_c0_potential <= 1000: 1}
{0 <= GALAMt2r_c0_revbin <= 1: -1000, 0 <= GALAMt2r_c0 <= 1000: 1}
{0 <= GALAMt2r_c0_revbin <= 1: 1000, 0 <= GALAMt2r_c0_reverse_06a6d <= 10

{0 <= INSH_c0_revbin <= 1: -1000, 0 <= INSH_c0 <= 1000: 1}
{0 <= INSH_c0_revbin <= 1: 1000, 0 <= INSH_c0_reverse_309c4 <= 0: 1}
{0 <= INSH_c0_revbin <= 1: 1000, 0 <= cpd00246_c0_potential <= 1000: -1, 0 <= cpd00001_c0_potential <= 1000: -1, 0 <= cpd00226_c0_potential <= 1000: 1, 0 <= cpd00105_c0_potential <= 1000: 1}
{0 <= INSt2_c0_revbin <= 1: -1000, 0 <= INSt2_c0 <= 1000: 1}
{0 <= INSt2_c0_revbin <= 1: 1000, 0 <= INSt2_c0_reverse_4673f <= 1000: 1}
{0 <= INSt2_c0_revbin <= 1: 1000, 0 <= cpd00246_e0_potential <= 1000: -1, 0 <= cpd00246_c0_potential <= 1000: 1, 0 <= cpd00067_c0_potential <= 1000: 1, 0 <= cpd00067_e0_potential <= 1000: -1}
{0 <= INULINASEe_c0_revbin <= 1: -1000, 0 <= INULINASEe_c0 <= 1000: 1}
{0 <= INULINASEe_c0_revbin <= 1: 1000, 0 <= INULINASEe_c0_reverse_7f909 <= 0: 1}
{0 <= INULINASEe_c0_revbin <= 1: 1000, 0 <= cpd11602_e0_potential <= 1000: -1, 0 <= cpd00001_e0_potential <= 1000: -25, 0 <= kestopt_e0_potential <= 1000: 1, 0 <= cpd00082_e0_potential <= 1000: 25}
{0 <

...

AMALT4_c0_revbinR: 1000.0*AMALT4_c0_revbin + 1.0*AMALT4_c0_reverse_b69a7 <= 1000
AMANNANASEe_c0_revbinR: 1000.0*AMANNANASEe_c0_revbin + 1.0*AMANNANASEe_c0_reverse_31df2 <= 1000
AMAOTr_c0_revbinR: 1000.0*AMAOTr_c0_revbin + 1.0*AMAOTr_c0_reverse_031d8 <= 1000
AMMQLT10_c0_revbinR: 1000.0*AMMQLT10_c0_revbin + 1.0*AMMQLT10_c0_reverse_13173 <= 1000
AMMQLT11_c0_revbinR: 1000.0*AMMQLT11_c0_revbin + 1.0*AMMQLT11_c0_reverse_c910e <= 1000
AMMQLT7_c0_revbinR: 1000.0*AMMQLT7_c0_revbin + 1.0*AMMQLT7_c0_reverse_8dc66 <= 1000
AMMQLT8_c0_revbinR: 1000.0*AMMQLT8_c0_revbin + 1.0*AMMQLT8_c0_reverse_1f420 <= 1000
AMMQLT9_c0_revbinR: 1000.0*AMMQLT9_c0_revbin + 1.0*AMMQLT9_c0_reverse_d3f54 <= 1000
AMMQT8__DASH__2_c0_revbinR: 1000.0*AMMQT8__DASH__2_c0_revbin + 1.0*AMMQT8__DASH__2_c0_reverse_c1969 <= 1000
AMPN_c0_revbinR: 1000.0*AMPN_c0_revbin + 1.0*AMPN_c0_reverse_58cec <= 1000
AMPTASECG_c0_revbinR: 1000.0*AMPTASECG_c0_revbin + 1.0*AMPTASECG_c0_reverse_949b7 <= 1000
AMPTASEPG_c0_revbinR: 1000.0*AMPTASEPG_c0_r

G5SADs_c0_revbinR: 1000.0*G5SADs_c0_revbin + 1.0*G5SADs_c0_reverse_45de9 <= 1000
G5SD_c0_revbinR: 1000.0*G5SD_c0_revbin + 1.0*G5SD_c0_reverse_2ed43 <= 1000
G6PBDH_c0_revbinR: 1000.0*G6PBDH_c0_revbin + 1.0*G6PBDH_c0_reverse_67f7e <= 1000
G6PDA_c0_revbinR: 1000.0*G6PDA_c0_revbin + 1.0*G6PDA_c0_reverse_8f11a <= 1000
G6PDH2r_c0_revbinR: 1000.0*G6PDH2r_c0_revbin + 1.0*G6PDH2r_c0_reverse_ca107 <= 1000
G6PI_c0_revbinR: 1000.0*G6PI_c0_revbin + 1.0*G6PI_c0_reverse_c2968 <= 1000
G6PI3_c0_revbinR: 1000.0*G6PI3_c0_revbin + 1.0*G6PI3_c0_reverse_f7ee2 <= 1000
GACPCD_c0_revbinR: 1000.0*GACPCD_c0_revbin + 1.0*GACPCD_c0_reverse_f8b96 <= 1000
GALAM6PDA_c0_revbinR: 1000.0*GALAM6PDA_c0_revbin + 1.0*GALAM6PDA_c0_reverse_cfac8 <= 1000
GALAMt2r_c0_revbinR: 1000.0*GALAMt2r_c0_revbin + 1.0*GALAMt2r_c0_reverse_06a6d <= 1000
GALASE1e_c0_revbinR: 1000.0*GALASE1e_c0_revbin + 1.0*GALASE1e_c0_reverse_0aa74 <= 1000
GALASE__DASH__HSe_c0_revbinR: 1000.0*GALASE__DASH__HSe_c0_revbin + 1.0*GALASE__DASH__HSe_c0_reverse_8a3

MMCD_c0_revbinR: 1000.0*MMCD_c0_revbin + 1.0*MMCD_c0_reverse_33a63 <= 1000
MME_c0_revbinR: 1000.0*MME_c0_revbin + 1.0*MME_c0_reverse_610ea <= 1000
MMM2r_c0_revbinR: 1000.0*MMM2r_c0_revbin + 1.0*MMM2r_c0_reverse_067c7 <= 1000
MNNH_c0_revbinR: 1000.0*MNNH_c0_revbin + 1.0*MNNH_c0_reverse_f4182 <= 1000
MNt2_c0_revbinR: 1000.0*MNt2_c0_revbin + 1.0*MNt2_c0_reverse_17f32 <= 1000
MOAT__DASH__BT_c0_revbinR: 1000.0*MOAT__DASH__BT_c0_revbin + 1.0*MOAT__DASH__BT_c0_reverse_ce970 <= 1000
MOHMT_c0_revbinR: 1000.0*MOHMT_c0_revbin + 1.0*MOHMT_c0_reverse_75808 <= 1000
MTAN_c0_revbinR: 1000.0*MTAN_c0_revbin + 1.0*MTAN_c0_reverse_e2121 <= 1000
MTHFC_c0_revbinR: 1000.0*MTHFC_c0_revbin + 1.0*MTHFC_c0_reverse_48112 <= 1000
MTHFD_c0_revbinR: 1000.0*MTHFD_c0_revbin + 1.0*MTHFD_c0_reverse_742d2 <= 1000
MTHFR2_c0_revbinR: 1000.0*MTHFR2_c0_revbin + 1.0*MTHFR2_c0_reverse_6f1cc <= 1000
MTHFR3_c0_revbinR: 1000.0*MTHFR3_c0_revbin + 1.0*MTHFR3_c0_reverse_c1f2b <= 1000
MTHGXLt_c0_revbinR: 1000.0*MTHGXLt_c0_revbin + 1.

SUCCt2r_c0_revbinR: 1000.0*SUCCt2r_c0_revbin + 1.0*SUCCt2r_c0_reverse_c2489 <= 1000
SUCD1_c0_revbinR: 1000.0*SUCD1_c0_revbin + 1.0*SUCD1_c0_reverse_489b7 <= 1000
SUCDi_c0_revbinR: 1000.0*SUCDi_c0_revbin + 1.0*SUCDi_c0_reverse_4f69e <= 1000
SUCOAS_c0_revbinR: 1000.0*SUCOAS_c0_revbin + 1.0*SUCOAS_c0_reverse_0e84c <= 1000
SUCORNTC_c0_revbinR: 1000.0*SUCORNTC_c0_revbin + 1.0*SUCORNTC_c0_reverse_d7027 <= 1000
SUCR_c0_revbinR: 1000.0*SUCR_c0_revbin + 1.0*SUCR_c0_reverse_c0dc4 <= 1000
SUCRt2_c0_revbinR: 1000.0*SUCRt2_c0_revbin + 1.0*SUCRt2_c0_reverse_a1657 <= 1000
T2DECAI_c0_revbinR: 1000.0*T2DECAI_c0_revbin + 1.0*T2DECAI_c0_reverse_9e511 <= 1000
TAGURr_c0_revbinR: 1000.0*TAGURr_c0_revbin + 1.0*TAGURr_c0_reverse_70fbf <= 1000
TALA_c0_revbinR: 1000.0*TALA_c0_revbin + 1.0*TALA_c0_reverse_95925 <= 1000
TAURtex2_c0_revbinR: 1000.0*TAURtex2_c0_revbin + 1.0*TAURtex2_c0_reverse_513be <= 1000
TCHOLAH_c0_revbinR: 1000.0*TCHOLAH_c0_revbin + 1.0*TCHOLAH_c0_reverse_33f07 <= 1000
TCHOLAt2_c0_revbinR: 1000

EX_fucneulacnhx_e0_revbinR: 1000.0*EX_fucneulacnhx_e0_revbin + 1.0*EX_fucneulacnhx_e0_reverse_3d97d <= 1000
EX_cpd00239_e0_revbinR: 1000.0*EX_cpd00239_e0_revbin + 1.0*EX_cpd00239_e0_reverse_32fd0 <= 1000
EX_arabinan101_e0_revbinR: 1000.0*EX_arabinan101_e0_revbin + 1.0*EX_arabinan101_e0_reverse_df880 <= 1000
EX_cpd01861_e0_revbinR: 1000.0*EX_cpd01861_e0_revbin + 1.0*EX_cpd01861_e0_reverse_f3820 <= 1000
EX_isoval_e0_revbinR: 1000.0*EX_isoval_e0_revbin + 1.0*EX_isoval_e0_reverse_ed6de <= 1000
EX_cpd00875_e0_revbinR: 1000.0*EX_cpd00875_e0_revbin + 1.0*EX_cpd00875_e0_reverse_a92b6 <= 1000
EX_cpd00396_e0_revbinR: 1000.0*EX_cpd00396_e0_revbin + 1.0*EX_cpd00396_e0_reverse_d882a <= 1000
EX_hspg_degr_11_e0_revbinR: 1000.0*EX_hspg_degr_11_e0_revbin + 1.0*EX_hspg_degr_11_e0_reverse_84f42 <= 1000
EX_core4_e0_revbinR: 1000.0*EX_core4_e0_revbin + 1.0*EX_core4_e0_reverse_eeaf5 <= 1000
EX_cpd00065_e0_revbinR: 1000.0*EX_cpd00065_e0_revbin + 1.0*EX_cpd00065_e0_reverse_0c855 <= 1000
EX_hspg_degr_14_e0_rev

In [6]:
'''simple_thermo = thermodynamic_API_KBase_01.SimpleThermoPkg(model, object = model.reactions.get_by_id('PFK'), thermo_reactions = reactions_dict, thermo_compounds = compounds_dict)

for item, value in simple_thermo.thermo_compounds.items(): 
    print(item, ': ', value)'''

"simple_thermo = thermodynamic_API_KBase_01.SimpleThermoPkg(model, object = model.reactions.get_by_id('PFK'), thermo_reactions = reactions_dict, thermo_compounds = compounds_dict)\n\nfor item, value in simple_thermo.thermo_compounds.items(): \n    print(item, ': ', value)"

# Full thermo functionality

The thermodynamic perspectives of each object metabolite are determined. The class calculates the total energy for each metabolite, which is the sum of each Gibbs potential, concentration potential, and electrochemical potential.

In [7]:
import thermodynamic_API_KBase_01
model = kbase.get_from_ws('E_iAH991V2', 93832)

announcement = 'Full Thermo'
print('\n', announcement, '\n', '='*len(announcement))


#--------------------------- __init__ ---------------------------------
print('\n+ __init__')

full_thermo = thermodynamic_API_KBase_01.FullThermoPkg(model, thermo_compounds = reactions, thermodynamics_data_type = 'kbase')

print('model used: ', full_thermo.model)
print('instance name: ', full_thermo.name)
print('variable types: ', full_thermo.variable_types)
print('constraint types: ', full_thermo.constraint_types)


#------------------------- build_constraint --------------------------
print('\n+ build_constraint')
print('constraints before: ', full_thermo.constraints)

full_thermo.build_constraint(obj = model.reactions.get_by_id('12ETHDt_c0'))

print('constraints after: ', full_thermo.constraints)
#full_thermo.model.constraints


#------------------------- build_package -----------------------------
print('\n+ build_package')
print('constraints before: ', full_thermo.constraints)

full_thermo.build_package(view_errors = False)

constraints = []
for constraint in full_thermo.constraints:
    for datam in full_thermo.constraints[constraint]:
        constraints.append(str(datam))
print('constraints after: ', len(constraints))


#------------------------- write_lp_file -----------------------------
from datetime import date
import os 

print('\n+ write_lp_file')
export_filename = 'Full Thermo KBase individual model test'
import_iteration = 0
filename = '{}_{}_{}.lp'.format(date.today(), export_filename, import_iteration)
print('LP file existing before: ', os.path.exists(filename))

full_thermo.write_lp_file(model = model, export_filename = export_filename)

print('LP file existing after: ', os.path.exists(filename)) 
#os.remove(filename)

#------------------------ revert_to_original --------------------------
model = simple_thermo.revert_to_original(kbase = kbase, kbase_model_id = 'E_iAH991V2', workspace_id = 93832)


 Full Thermo 

+ __init__
model used:  Bacteroides_thetaiotaomicron_VPI-5482.fbamdl.23
instance name:  full_thermo
variable types:  {'revbin': {}, 'forv': {}, 'revv': {}, 'ln_concentration': {}, 'potential': {}}
constraint types:  {'concentration_potential': 'metabolite', 'electrochemical_potential': 'metabolite', 'full_thermo': 'metabolite', 'revbinF': {}, 'revbinR': {}, 'simple_thermo': {}}

+ build_constraint
constraints before:  {'concentration_potential': {}, 'electrochemical_potential': {}, 'full_thermo': {}, 'revbinF': {}, 'revbinR': {}, 'simple_thermo': {}}
{0 <= 12ETHDt_c0_revbin <= 1: -1000, 0 <= 12ETHDt_c0 <= 1000: 1}
{0 <= 12ETHDt_c0_revbin <= 1: 1000, 0 <= 12ETHDt_c0_reverse_af890 <= 1000: 1}
{0.001 <= 12ethd_c0_ln_concentration <= 20: 1, 0.001 <= 12ethd_e0_ln_concentration <= 20: -1, 0 <= 12ETHDt_c0_revbin <= 1: 1000}
constraints after:  {'concentration_potential': {}, 'electrochemical_potential': {}, 'full_thermo': {'12ETHDt_c0': <optlang.cplex_interface.Constraint obje

{0 <= ADK2_c0_revbin <= 1: -1000, 0 <= ADK2_c0 <= 1000: 1}
{0 <= ADK2_c0_revbin <= 1: 1000, 0 <= ADK2_c0_reverse_e69ea <= 1000: 1}
ERROR: A cpd00018_c0 variable already exists, 0 <= cpd00018_c0_potential <= 1000.
ERROR: A cpd00008_c0 variable already exists, 0 <= cpd00008_c0_potential <= 1000.
ERROR: A cpd00012_c0 variable already exists, 0 <= cpd00012_c0_potential <= 1000.
{0 <= ADK2_c0_revbin <= 1: 1000, 0 <= cpd00018_c0_potential <= 1000: -1, 0 <= cpd00008_c0_potential <= 1000: 1, 0 <= cpd00012_c0_potential <= 1000: 1, 0 <= cpd00421_c0_potential <= 1000: -1}
{0 <= ADNUC_c0_revbin <= 1: -1000, 0 <= ADNUC_c0 <= 1000: 1}
{0 <= ADNUC_c0_revbin <= 1: 1000, 0 <= ADNUC_c0_reverse_7c848 <= 0: 1}
ERROR: A cpd00001_c0 variable already exists, 0 <= cpd00001_c0_potential <= 1000.
ERROR: A cpd00128_c0 variable already exists, 0 <= cpd00128_c0_potential <= 1000.
{0 <= ADNUC_c0_revbin <= 1: 1000, 0 <= cpd00001_c0_potential <= 1000: -1, 0 <= cpd00128_c0_potential <= 1000: 1, 0 <= cpd00182_c0_potent

{0 <= CLPNSI16_c0_revbin <= 1: -1000, 0 <= CLPNSI16_c0 <= 1000: 1}
{0 <= CLPNSI16_c0_revbin <= 1: 1000, 0 <= CLPNSI16_c0_reverse_91983 <= 1000: 1}
ERROR: A cpd00100_c0 variable already exists, 0 <= cpd00100_c0_potential <= 1000.
{0 <= CLPNSI16_c0_revbin <= 1: 1000, 0 <= pgi16_c0_potential <= 1000: -2, 0 <= clpni16_c0_potential <= 1000: 1, 0 <= cpd00100_c0_potential <= 1000: 1}
{0 <= CLPNSI17_c0_revbin <= 1: -1000, 0 <= CLPNSI17_c0 <= 1000: 1}
{0 <= CLPNSI17_c0_revbin <= 1: 1000, 0 <= CLPNSI17_c0_reverse_0e1e2 <= 1000: 1}
ERROR: A cpd00100_c0 variable already exists, 0 <= cpd00100_c0_potential <= 1000.
{0 <= CLPNSI17_c0_revbin <= 1: 1000, 0 <= clpni17_c0_potential <= 1000: 1, 0 <= cpd00100_c0_potential <= 1000: 1, 0 <= pgi17_c0_potential <= 1000: -2}
{0 <= CMLDC_c0_revbin <= 1: -1000, 0 <= CMLDC_c0 <= 1000: 1}
{0 <= CMLDC_c0_revbin <= 1: 1000, 0 <= CMLDC_c0_reverse_5bb34 <= 0: 1}
ERROR: A cpd00067_c0 variable already exists, 0 <= cpd00067_c0_potential <= 1000.
ERROR: A cpd00011_c0 varia

{0 <= ETOHt_c0_revbin <= 1: -1000, 0 <= ETOHt_c0 <= 1000: 1}
{0 <= ETOHt_c0_revbin <= 1: 1000, 0 <= ETOHt_c0_reverse_2d843 <= 1000: 1}
ERROR: A cpd00363_c0 variable already exists, 0 <= cpd00363_c0_potential <= 1000.
{0 <= ETOHt_c0_revbin <= 1: 1000, 0 <= cpd00363_c0_potential <= 1000: 1, 0 <= cpd00363_e0_potential <= 1000: -1}
{0 <= F1PGT_c0_revbin <= 1: -1000, 0 <= F1PGT_c0 <= 1000: 1}
{0 <= F1PGT_c0_revbin <= 1: 1000, 0 <= F1PGT_c0_reverse_f3131 <= 1000: 1}
ERROR: A cpd00272_c0 variable already exists, 0 <= cpd00272_c0_potential <= 1000.
ERROR: A cpd00012_c0 variable already exists, 0 <= cpd00012_c0_potential <= 1000.
ERROR: A cpd00038_c0 variable already exists, 0 <= cpd00038_c0_potential <= 1000.
ERROR: A cpd00067_c0 variable already exists, 0 <= cpd00067_c0_potential <= 1000.
{0 <= F1PGT_c0_revbin <= 1: 1000, 0 <= cpd00272_c0_potential <= 1000: 1, 0 <= cpd01912_c0_potential <= 1000: -1, 0 <= cpd00012_c0_potential <= 1000: 1, 0 <= cpd00038_c0_potential <= 1000: -1, 0 <= cpd00067_c

{0 <= GLYBabc_c0_revbin <= 1: 1000, 0 <= cpd00001_c0_potential <= 1000: -1, 0 <= cpd00008_c0_potential <= 1000: 1, 0 <= cpd00002_c0_potential <= 1000: -1, 0 <= cpd00540_c0_potential <= 1000: 1, 0 <= cpd00067_c0_potential <= 1000: 1, 0 <= cpd00009_c0_potential <= 1000: 1, 0 <= cpd00540_e0_potential <= 1000: -1}
{0 <= GLYCK_c0_revbin <= 1: -1000, 0 <= GLYCK_c0 <= 1000: 1}
{0 <= GLYCK_c0_revbin <= 1: 1000, 0 <= GLYCK_c0_reverse_f42d7 <= 0: 1}
ERROR: A cpd00008_c0 variable already exists, 0 <= cpd00008_c0_potential <= 1000.
ERROR: A cpd00002_c0 variable already exists, 0 <= cpd00002_c0_potential <= 1000.
ERROR: A cpd00067_c0 variable already exists, 0 <= cpd00067_c0_potential <= 1000.
{0 <= GLYCK_c0_revbin <= 1: 1000, 0 <= cpd00169_c0_potential <= 1000: 1, 0 <= cpd00008_c0_potential <= 1000: 1, 0 <= cpd00223_c0_potential <= 1000: -1, 0 <= cpd00002_c0_potential <= 1000: -1, 0 <= cpd00067_c0_potential <= 1000: 1}
{0 <= GLYCOe_c0_revbin <= 1: -1000, 0 <= GLYCOe_c0 <= 1000: 1}
{0 <= GLYCOe_c0_

...

{0.001 <= cpd00027_c0_ln_concentration <= 20: 1, 0.001 <= cpd00179_c0_ln_concentration <= 20: -1, 0.001 <= cpd01329_c0_ln_concentration <= 20: 1, 0.001 <= cpd15495_c0_ln_concentration <= 20: -1, 0 <= AMALT3_c0_revbin <= 1: 1000}
{0.001 <= cpd00027_c0_ln_concentration <= 20: 1, 0.001 <= cpd00179_c0_ln_concentration <= 20: -1, 0.001 <= cpd15494_c0_ln_concentration <= 20: 1, 0.001 <= cpd01329_c0_ln_concentration <= 20: -1, 0 <= AMALT4_c0_revbin <= 1: 1000}
{0.001 <= amannan140_e0_ln_concentration <= 20: -1, 0.001 <= cpd00001_e0_ln_concentration <= 20: -139, 0.001 <= cpd00138_e0_ln_concentration <= 20: 140, 0 <= AMANNANASEe_c0_revbin <= 1: 1000}
{0.001 <= cpd00017_c0_ln_concentration <= 20: -1, 0.001 <= cpd02701_c0_ln_concentration <= 20: 1, 0.001 <= cpd00764_c0_ln_concentration <= 20: 1, 0.001 <= cpd00800_c0_ln_concentration <= 20: -1, 0 <= AMAOTr_c0_revbin <= 1: 1000}
{0.001 <= cpd00017_c0_ln_concentration <= 20: -1, 0.001 <= 2dmmql10_c0_ln_concentration <= 20: -1, 0.001 <= cpd00019_c0_l

{0.001 <= cpd00350_c0_ln_concentration <= 20: -1, 0.001 <= ggdp_c0_ln_concentration <= 20: 1, 0.001 <= cpd00113_c0_ln_concentration <= 20: -1, 0.001 <= cpd00012_c0_ln_concentration <= 20: 1, 0 <= FRTT_c0_revbin <= 1: 1000}
{0.001 <= cpd00001_c0_ln_concentration <= 20: -1, 0.001 <= cpd00082_c0_ln_concentration <= 20: 2, 0.001 <= levanb_c0_ln_concentration <= 20: -1, 0 <= FRUASE3_c0_revbin <= 1: 1000}
{0.001 <= cpd00082_c0_ln_concentration <= 20: 1, 0.001 <= cpd00067_c0_ln_concentration <= 20: 1, 0.001 <= cpd00082_e0_ln_concentration <= 20: -1, 0.001 <= cpd00067_e0_ln_concentration <= 20: -1, 0 <= FRUt2r_c0_revbin <= 1: 1000}
{0.001 <= cpd00013_c0_ln_concentration <= 20: 1, 0.001 <= cpd00347_c0_ln_concentration <= 20: 1, 0.001 <= cpd00067_c0_ln_concentration <= 20: -2, 0.001 <= 5forthf_c0_ln_concentration <= 20: -1, 0 <= FTCD__DASH__r_c0_revbin <= 1: 1000}
{0.001 <= cpd02197_c0_ln_concentration <= 20: -1, 0.001 <= cpd00008_c0_ln_concentration <= 20: 1, 0.001 <= cpd00002_c0_ln_concentrati

{0.001 <= mantr_e0_ln_concentration <= 20: 1, 0.001 <= cpd00001_e0_ln_concentration <= 20: -1, 0.001 <= manttr_e0_ln_concentration <= 20: -1, 0.001 <= cpd00138_e0_ln_concentration <= 20: 1, 0 <= MANA3e_c0_revbin <= 1: 1000}
{0.001 <= cpd00001_c0_ln_concentration <= 20: -1, 0.001 <= cpd00108_c0_ln_concentration <= 20: 1, 0.001 <= cpd03198_c0_ln_concentration <= 20: 1, 0.001 <= mantr_c0_ln_concentration <= 20: -1, 0 <= MANA4_c0_revbin <= 1: 1000}
{0.001 <= cpd00001_e0_ln_concentration <= 20: -1, 0.001 <= manttr_e0_ln_concentration <= 20: 1, 0.001 <= manpt_e0_ln_concentration <= 20: -1, 0.001 <= cpd00138_e0_ln_concentration <= 20: 1, 0 <= MANA4e_c0_revbin <= 1: 1000}
{0.001 <= cpd00672_c0_ln_concentration <= 20: 1, 0.001 <= cpd00003_c0_ln_concentration <= 20: -1, 0.001 <= cpd00004_c0_ln_concentration <= 20: 1, 0.001 <= cpd00403_c0_ln_concentration <= 20: -1, 0.001 <= cpd00067_c0_ln_concentration <= 20: 1, 0 <= MANAO_c0_revbin <= 1: 1000}
{0.001 <= mantr_e0_ln_concentration <= 20: -1, 0.00

{0.001 <= cpd00048_c0_ln_concentration <= 20: -1, 0.001 <= cpd00002_c0_ln_concentration <= 20: -1, 0.001 <= cpd00012_c0_ln_concentration <= 20: 1, 0.001 <= cpd00067_c0_ln_concentration <= 20: -1, 0.001 <= cpd00193_c0_ln_concentration <= 20: 1, 0 <= SADT_c0_revbin <= 1: 1000}
{0.001 <= cpd00001_c0_ln_concentration <= 20: -1, 0.001 <= cpd00002_c0_ln_concentration <= 20: -1, 0.001 <= cpd00012_c0_ln_concentration <= 20: 1, 0.001 <= cpd00031_c0_ln_concentration <= 20: 1, 0.001 <= cpd00009_c0_ln_concentration <= 20: 1, 0.001 <= cpd00048_c0_ln_concentration <= 20: -1, 0.001 <= cpd00038_c0_ln_concentration <= 20: -1, 0.001 <= cpd00193_c0_ln_concentration <= 20: 1, 0 <= SADT2_c0_revbin <= 1: 1000}
{0.001 <= cpd00024_c0_ln_concentration <= 20: -1, 0.001 <= cpd00658_c0_ln_concentration <= 20: -1, 0.001 <= cpd00067_c0_ln_concentration <= 20: -1, 0.001 <= 2s5epchc_c0_ln_concentration <= 20: 1, 0.001 <= cpd00011_c0_ln_concentration <= 20: 1, 0 <= SEPHCHS_c0_revbin <= 1: 1000}
{0.001 <= cpd00722_c0_l

{0.001 <= hspg_degr_15_e0_ln_concentration <= 20: -1, 0 <= EX_hspg_degr_15_e0_revbin <= 1: 1000}
{0.001 <= gncore1_e0_ln_concentration <= 20: -1, 0 <= EX_gncore1_e0_revbin <= 1: 1000}
{0.001 <= arabttr_e0_ln_concentration <= 20: -1, 0 <= EX_arabttr_e0_revbin <= 1: 1000}
{0.001 <= cpd00703_e0_ln_concentration <= 20: -1, 0 <= EX_cpd00703_e0_revbin <= 1: 1000}
{0.001 <= cpd00011_e0_ln_concentration <= 20: -1, 0 <= EX_cpd00011_e0_revbin <= 1: 1000}
{0.001 <= cspg_a_degr_e0_ln_concentration <= 20: -1, 0 <= EX_cspg_a_degr_e0_revbin <= 1: 1000}
{0.001 <= pe_cer1_bt_e0_ln_concentration <= 20: -1, 0 <= EX_pe_cer1_bt_e0_revbin <= 1: 1000}
{0.001 <= hspg_degr_1_e0_ln_concentration <= 20: -1, 0 <= EX_hspg_degr_1_e0_revbin <= 1: 1000}
{0.001 <= cpd00036_e0_ln_concentration <= 20: -1, 0 <= EX_cpd00036_e0_revbin <= 1: 1000}
{0.001 <= cpd00254_e0_ln_concentration <= 20: -1, 0 <= EX_cpd00254_e0_revbin <= 1: 1000}
{0.001 <= cpd00105_e0_ln_concentration <= 20: -1, 0 <= EX_cpd00105_e0_revbin <= 1: 1000}
{

# Brainstorming

In [8]:
display(dir(modelseed))
import re

for metabolite in model.metabolites:
    print(metabolite.name)
    id_name = re.sub('(_.$)', '', metabolite.id)
    print(id_name)
    print(modelseed.get_seed_compound_by_alias(database = modelseed, cpd_id = id_name))
    modelseed.get_seed_compound_by_alias(database = modelseed, cpd_id = id_name)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'compound_aliases',
 'compound_structures',
 'compounds',
 'get_attribute',
 'get_formula',
 'get_mapping_external_to_seed',
 'get_name',
 'get_non_obsolete',
 'get_seed_compound',
 'get_seed_compound_by_alias',
 'get_seed_reaction',
 'get_seed_reaction_by_alias',
 'reaction_aliases',
 'reaction_ecs',
 'reactions',
 'summary']

12ethd_e0
12ethd_e0


IndexError: list index out of range

In [ ]:
import json
modelseed_compounds = json(open('{}_APF_described compounds in ModelSEED_01.json'))

# convert the KBase model into a COBRA model
genome_json = kbase.get_object('GCF_000005845.2', ws)
info = kbase.get_object_info('GCF_000005845.2', ws)
genome = KBaseGenome(genome_json, info)

b = cobrakbase.core.converters.KBaseFBAModelToCobraBuilder(fbamodel)
#b.with_media() # OPTIONAL include media def
b.with_genome(genome) # OPTIONAL include genome annotation in the cobra model
model = b.build()



if element in described_compounds_list:
    for compound, aliases in nist_compounds.items():
        if element.lower() == compound.lower() or element in aliases['alternative names']:
            reactants.append(coefficient + aliases['ModelSEED ID'])
            
            
reaction_ids = []
for reaction in complete_reactions:
    if reaction not in reaction_id_pairs.values():
        reaction_id = 'TECRDB{}'.format(id_number)
        reaction_id_pairs[reaction_id] = reaction
        id_number += 1
    else:
        index = list(reaction_id_pairs.values()).index(reaction)
        reaction_id = list(reaction_id_pairs.keys())[index]
        print('--> {} is a redundant reaction.'.format(reaction))

    reaction_ids.append(reaction_id)

In [ ]:
import cobrakbase
# import the E. coli BiGG COBRA model
import cobra
bigg_model_path = '..\COBRA function scripts\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)

os.environ["HOME"] = 'C:\\Users\\Andrew Freiburger\\Dropbox\\My PC (DESKTOP-M302P50)\\Documents\\UVic Civil Engineering\\Internships\\Agronne\\cobrakbase'
import cobrakbase
token = 'SF3CW6PH4CKWSRMOA3JQPSGC6YERTRSF'
kbase = cobrakbase.KBaseAPI(token)
object_json = kbase.get_object('E_iAH991V2', 93832)

#dir(cobrakbase.KBaseFBAModelToCobraBuilder.convert_modelcompound)

'''kbase_model = cobrakbase.KBaseFBAModelToCobraBuilder(model)
for metabolite in model.metabolites:
    kbase_model.convert_modelcompound(metabolite = metabolite)'''
    
    
dir(kbase_model)

In [ ]:
import cobrakbase
# import the E. coli BiGG COBRA model
'''import cobra
bigg_model_path = '..\COBRA function scripts\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)'''

os.environ["HOME"] = 'C:\\Users\\Andrew Freiburger\\Dropbox\\My PC (DESKTOP-M302P50)\\Documents\\UVic Civil Engineering\\Internships\\Agronne\\cobrakbase'
import cobrakbase
token = 'SF3CW6PH4CKWSRMOA3JQPSGC6YERTRSF'
kbase = cobrakbase.KBaseAPI(token)

# the duplicate compound is removed from the KBase model 
object_json = kbase.get_object('E_iAH991V2', 93832)
unique_mcs = dict((x['id'], x) for x in object_json['modelcompounds']) 
object_json['modelcompounds'] = list(unique_mcs.values()) # update data without the duplicate id
kbase.save_object('E_iAH991V2', 'freiburgermsu:narrative_1624557251879', 'KBaseFBA.FBAModel', object_json) # saving the object back to KBase with id=E_iAH991V2 to the workspace freiburgermsu:narrative_1624557251879 assigning this type KBaseFBA.FBAModel and with data=object_json

#dir(cobrakbase.KBaseFBAModelToCobraBuilder.convert_modelcompound)
'''kbase_model = cobrakbase.KBaseFBAModelToCobraBuilder(model)
for metabolite in model.metabolites:
    kbase_model.convert_modelcompound(metabolite = metabolite)'''
    
    
#dir(cobrakbase.KBaseAPI)

kbase_model = kbase.get_from_ws('E_iAH991V2', 93832)
for metabolite in kbase_model.metabolites:
    print(metabolite)

In [ ]:
import cobrakbase
# import the E. coli BiGG COBRA model
'''import cobra
bigg_model_path = '..\COBRA function scripts\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)'''

os.environ["HOME"] = 'C:\\Users\\Andrew Freiburger\\Dropbox\\My PC (DESKTOP-M302P50)\\Documents\\UVic Civil Engineering\\Internships\\Agronne\\cobrakbase'
import cobrakbase
token = 'SF3CW6PH4CKWSRMOA3JQPSGC6YERTRSF'
kbase = cobrakbase.KBaseAPI(token)
model = kbase.get_from_ws('E_iAH991V2', 93832)
#object_json = kbase.get_object('E_iAH991V2', 93832)

#dir(cobrakbase.KBaseFBAModelToCobraBuilder.convert_modelcompound)

'''kbase_model = cobrakbase.KBaseFBAModelToCobraBuilder(model)
for metabolite in model.metabolites:
    kbase_model.convert_modelcompound(metabolite = metabolite)'''
    
    
#dir(cobrakbase.KBaseAPI)

'''for metabolite in model.metabolites:
    print(metabolite)'''
    
for reaction in model.reactions:
    print(reaction)
    
cobrakbase.core.kbasefba.fbamodel_builder.FBAModelBuilder.from_kbase_json(dict_data, info, None)